## Authors: Oguzhan Ugur, Shadman Ahmed
## Discovery of Frequent Itemsets and Association Rules


### Problem Formulation
Frequent itemsets is about finding sets of items that appear in number of baskets at the same time. This model is called "market-basket" model of data, where the realtionship between the "items" and "baskets" are important. <br>
<br>
In this homework, we were instructed to implement the apriori algorithm for finding frequent itemsets, with support at least s. We were also instructed to generate association rules between the frequent itemsets with at least confidence c and support s in a data set of sales transaction. 

### Dataset
The data used in this homework was a sale transaction dataset which included generated transactions(baskets) of hashed  items. 

### Apriori Algorithm
The apriori algorithm finds the frequent itemsets of the dataset by taking a minimum support value as input. This algorithm will generate a dictionary with the frequent itemsets as key and the index of baskets the items appear in as value of the dictionary. This dictionary will then be scanned in order to filter out the items with less support than s. The remaining items are then combined to create doubletons(itemsets with two elements). The doubletons will then be scanned and the doubletons with less support than s will be filtered out again. We chose to repeat this procedure to a chosen number of combinations k. So, it is possible to choose the size of the itemsets. 

#### Code
The apriori algorithm consist of 4 functions, <br>
- First, "all_items_list_of_occurances" that takes the dataset file as input and returns a dictionary. The key values of the dictionary will be the itemsets and the value are a list of corresponding baskets that the itemset appears in. 
- Second, "Scanner" that filters out the itemsets with less support than a chosen number s and returns a dictionary with the same structure as before. 
- Third, "combination" that creates the combinations between the filtered itemsets and returns a set of these combinations. 
- Fourth, "scan_for_combinations" that scans the set of combinations created on the previous function and returns a dictionary with the combined itemsets(new itemsets) as key and the value of this dictionary are the corresponding baskets that these combined items appear in. 

The Second,Third and Fourth points are repeated in order to create more combinations of these itemssets and see the most frequent items that appear together.

### Notice 
The Association rule is described below after the code of the Apriori Algorithm


In [1]:
import itertools as it
from collections import defaultdict

In [55]:
#Set variables
s = 2          #Support threshold
k = 3            #Maximum combinations
c = 0.8          #Confidence threshold

In [5]:
def all_items_list_of_occurances(file):
    
    all_items = defaultdict(set)

    with open(file, "r") as f:
        for trans_index,trans in enumerate(f):
            trans_clean = trans.split()

            for item in trans_clean:
                    all_items[frozenset([item])].add(trans_index)
        return(all_items)

In [6]:
def Scanner(all_items, s):

    C_k_scanned = {}
    rest = {}
    
    for item, occurance_list in all_items.items():
        if(len(occurance_list) >= s):
            C_k_scanned[item] = occurance_list


    return(C_k_scanned)
    

In [7]:
def combination(k_tons, single_tons,k):
    combination = {items.union(item) for items in k_tons.keys()
                                        for item in single_tons.keys()
                                          if len(items.union(item)) == k}
    
    return combination
    

In [8]:
def scan_for_combinations(combinations, singelton_dict):

    scaned_ck = {}
    for combination in combinations:
        all_occurances = (singelton_dict[frozenset([item])] for item in combination)
        occurances = set.intersection(*all_occurances)
        
        scaned_ck[combination] = occurances
    
    return(scaned_ck)

In [43]:
all_items = all_items_list_of_occurances("T10I4D5.dat")

L1 = Scanner(all_items, s)

C2 = combination(L1,L1,2)

C2 = scan_for_combinations(C2, L1)
print(C2)
print("----------")
L2 = Scanner(C2, s)

C3 = combination(L2,L1,3)

C3 = scan_for_combinations(C3, L1)
print(C3)
print("----------")
L3 = Scanner(C3, s)

print(L3)

{frozenset({'704', '825'}): {1, 3}, frozenset({'825', '834'}): {0, 1}, frozenset({'39', '834'}): {1}, frozenset({'704', '834'}): {1}, frozenset({'704', '39'}): {1, 3}, frozenset({'825', '39'}): {1, 3}}
----------
{frozenset({'825', '39', '834'}): {1}, frozenset({'704', '39', '834'}): {1}, frozenset({'704', '825', '834'}): {1}, frozenset({'704', '825', '39'}): {1, 3}}
----------
{frozenset({'704', '825', '39'}): {1, 3}}


In [56]:
def main(s,k): 
    all_items = all_items_list_of_occurances("T10I4D5.dat")
    singleton_list= Scanner(all_items, s)
    kton_supported=singleton_list
    
    k_dict = {}
    k_dict[1] = singleton_list
    for k in range(2,k+1):
        k_comb = combination(kton_supported,singleton_list,k)
        kton_value = scan_for_combinations(k_comb, singleton_list)
        kton_supported= Scanner(kton_value,s)
    
        k_dict[k] = kton_supported
    return(kton_supported,k_dict)




In [57]:
frequent_items_dict, k_dict= main(s,k)


In [58]:
#print frequent itemset with at least support s 

format = "{:<50}{:<10}"  
print(format.format("items" , "support"))
for item_sets in range(1,len(k_dict)+1):
    for items in k_dict[item_sets]:
        print(format.format(str(items)+ ": " 
                                   , str(len(k_dict[item_sets][items]))))

items                                             support   
frozenset({'825'}):                               3         
frozenset({'834'}):                               2         
frozenset({'39'}):                                2         
frozenset({'704'}):                               2         
frozenset({'704', '825'}):                        2         
frozenset({'825', '834'}):                        2         
frozenset({'704', '39'}):                         2         
frozenset({'825', '39'}):                         2         
frozenset({'704', '825', '39'}):                  2         


## Association Rules

The association rules algorithm generates rules about the contents of the transactions. Namely, if a transaction contains a collection of items then it is also likely to contain a particular item $j$. An association rule can be stated as following: {$i_1$, $i_2$ , , $i_k$}  $\rightarrow$ $j$ if the transaction contains items {$i_1$, $i_2$ , , $i_k$} then it is likely to also contain $j$. The confidence of this association rule is the fraction of {$i_1$, $i_2$ , , $i_k$, $j$} support number and the {$i_1$, $i_2$ , , $i_k$} support number.
 
 
The association rules algorithm consist of three functions,<br>
- First, The "create_frequent_item_list" function that returns a list of the frequent itemset produced by the apriori algorithm.

- Second, The "generate_association" function that takes the frequent itemset list and extracts two itemsets at a time, item_set1 and item_set2. If item_set1 is a subset of item_set2 then we create an association rule where we calculate the confidence score by using the function "confidence". Then we print out the association rules with confidence above $c$ threshold.

- Third, The "confidence" function takes item_set2 and extract the key from the frequent itemset list (which is the corresponding support number). And divide it by the support number of item_set1, which is a subset of item_set2.


In [59]:
def create_frequent_item_list(k_dict):
    L = []
    for k1 in range(1,len(k_dict)+1):
        for items in k_dict[k1]:
            L.append(items)
    return L


In [60]:
def generate_association(k_dict, conf_threshold = 0.7):
    
    L = create_frequent_item_list(k_dict)
    result = []
    
    format = "{:<30}{:<10}{:<40}{:<10}"    
    print(format.format("confidence denominator"," ","numerator = denom + item_below", "confidence score"))
    for i in range(len(L)):
        for j in range(i+1,len(L)):
            if set(L[i]).issubset(L[j]):
                conf = confidence(k_dict,L[i],L[j])
                if(conf >= conf_threshold):
                    conf = ("%.2f" % conf)
                    print(format.format(str(L[i]),"=>",str(L[j].difference(L[i])),conf))
                    associations = (str(L[i])+" => "+str(L[j])+" with confidence: "+ str(conf))
                    result.append(associations)
                    
    return result

In [61]:
def confidence(frequent_item_set,item_set1,item_set2):
    
    union = len(frequent_item_set[len(item_set2)][item_set2])
    sup_itemset = len(frequent_item_set[len(item_set1)][item_set1])
    
    conf = union/sup_itemset
    
    return conf   

In [62]:
result = generate_association(k_dict, c)

confidence denominator                  numerator = denom + item_below          confidence score
frozenset({'834'})            =>        frozenset({'825'})                      1.00      
frozenset({'39'})             =>        frozenset({'704'})                      1.00      
frozenset({'39'})             =>        frozenset({'825'})                      1.00      
frozenset({'39'})             =>        frozenset({'704', '825'})               1.00      
frozenset({'704'})            =>        frozenset({'825'})                      1.00      
frozenset({'704'})            =>        frozenset({'39'})                       1.00      
frozenset({'704'})            =>        frozenset({'825', '39'})                1.00      
frozenset({'704', '825'})     =>        frozenset({'39'})                       1.00      
frozenset({'704', '39'})      =>        frozenset({'825'})                      1.00      
frozenset({'825', '39'})      =>        frozenset({'704'})                      1.00